In [1]:
import pandas as pd
from ast import literal_eval
def return_liter_eval(x):
    try:
        return list(literal_eval(x))
    except ValueError:
        return None

In [2]:
import numpy as np

In [3]:
meta_df = pd.read_csv("./gutenberg_meta_df.csv")

In [4]:
print(meta_df.shape)
meta_df.head()

(10884, 30)


,Unnamed: 0,id,title,author,authoryearofbirth,authoryearofdeath,language,downloads,subjects,type,...,genre_thriller,genre_western,genre_horror,genre_mystery,genre_crime,genre_history,genre_periodicals,genre_christian,genre_other,log_downloads
0,0,PG10002,The House on the Borderland,"Hodgson, William Hope",1877.0,1918.0,['en'],628,{'Science fiction'},Text,...,0,0,0,0,0,0,0,0,0,6.442540
1,1,PG10008,The Mystery,"White, Stewart Edward",1873.0,1946.0,['en'],92,{'Science fiction'},Text,...,0,0,0,0,0,0,0,0,0,4.521789
2,2,PG10021,Tenterhooks,"Leverson, Ada",1862.0,1933.0,['en'],60,"{'Man-woman relationships -- Fiction', 'London...",Text,...,0,0,0,0,0,0,0,0,1,4.094345
3,3,PG10025,Gaslight Sonatas,"Hurst, Fannie",1889.0,1968.0,['en'],45,"{'Slums -- Fiction', 'City and town life -- Fi...",Text,...,0,0,0,0,0,0,0,0,1,3.806662
4,4,PG10027,"The Triple Alliance, Its Trials and Triumphs","Avery, Harold",1867.0,1943.0,['en'],46,{'Schools -- Juvenile fiction'},Text,...,0,0,0,0,0,0,0,0,1,3.828641


In [5]:
import os
from tqdm import tqdm
from IPython.display import clear_output

In [6]:
books = meta_df.id.unique()

In [7]:
from glob import glob

In [8]:
len(glob("../../gutenberg_standard/new_analysis/proc_text_new/*_text.txt"))

10884

In [9]:
import seaborn as sns

In [10]:
from booknlp.booknlp import BookNLP
model_params={
                "pipeline":"entity,quote,supersense,event,coref", 
                "model":"small"
                }

booknlp=BookNLP("en", model_params)

/home/xjgong/anaconda3/envs/narrative/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/xjgong/anaconda3/envs/narrative/lib/python3.11/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/xjgong/anaconda3/envs/narrative/lib/python3.11/site-packages/torch/cuda/__init__.py:991: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  r = torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
/home/xjgong/anaconda3/envs/narrative/lib/python3.11/site-packages/networkx/

using device cpu
{'pipeline': 'entity,quote,supersense,event,coref', 'model': 'small'}
--- startup: 1.884 seconds ---


 69%|████████████████████████████████████████████████████████████████████████████████████████▍                                       | 7518/10884 [02:00<00:47, 70.18it/s]

In [11]:
books.shape

(10884,)

In [12]:
# these files are failed for NLP analysis for too large files, 
# to avoid break the loop, 
# skip these books
skip = ["PG17530", "PG3254", "PG4500", "PG5600", "PG5400"]

In [13]:
len(glob("../../gutenberg_standard/new_analysis/book_nlp_output_small/*/*.book"))

10154

In [14]:

# file_size = []
for b in tqdm(books):
    inputFile = "../../gutenberg_standard/new_analysis/proc_text_new/{}_text.txt".format(b)
    outputDir = "../../gutenberg_standard/new_analysis/book_nlp_output_small/{}/".format(b)
    idd = b
    
    if not os.path.exists(outputDir):
        os.mkdir(outputDir)

    if (not os.path.exists(outputDir + "{}.book".format(b))) and (b not in skip):
        print("Processing book: {} ->> file size: {} mb".format(b, (os.path.getsize(inputFile)/ (1024 * 1024))))
        booknlp.process(inputFile, outputDir, idd)
        
        clear_output(wait=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10884/10884 [14:02:15<00:00,  4.64s/it]


In [17]:
book_nlp_outputs = glob("../../gutenberg_standard/new_analysis/book_nlp_output_small/*")

In [18]:
len(book_nlp_outputs)

10884